<a href="https://colab.research.google.com/github/Q815S/NE/blob/main/NE7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai
!pip install SpeechRecognition pydub gtts

import google.generativeai as genai
import speech_recognition as sr
import pathlib,textwrap, os, requests,re


from IPython.display import display, Audio, display, Markdown
from google.colab import userdata, files
from pydub import AudioSegment
from gtts import gTTS

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-pro-exp-0801')

In [7]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def recognize_speech(file_path):
  try:
    sound = AudioSegment.from_file(file_path, format='m4a')
    sound.export('audio.wav', format='wav')

    r = sr.Recognizer()
    with sr.AudioFile('audio.wav') as source:
      audio = r.record(source)
    VR = r.recognize_google(audio, language='ko')
    return VR

  except sr.UnknownValueError:
    print("음성을 인식할 수 없습니다.")
    return None

  except sr.RequestError as e:
    print(f"음성인식 시스템이 응답하지 않습니다. 잠시 후 다시 시도해주십시오.; {e}")
    return None

  finally:
    os.remove('audio.wav')

In [ ]:
while True:
  print('듣고 있어요.')
  myfiles = files.upload()
  VR = recognize_speech('audio.m4a')

  if VR:
    response = model.generate_content(VR)
    tts = gTTS(text=response.text, lang='ko')
    tts.save('reply.mp3')
    voice = Audio('reply.mp3', autoplay=True)
    print('인식된 음성:',VR)
    markdown_text = to_markdown(response.text)
    display(voice)
    display(markdown_text)
    os.remove('audio.m4a')

  if VR == '고마워 잘가':
    print("안녕히 가세요!")
    break